In [88]:
import os
import json
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [89]:
dotenv.load_dotenv()

True

In [90]:
language_model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name=os.getenv("OPENAI_CHAT_MODEL_NAME"),
    temperature=float(os.getenv("OPENAI_CHAT_MODEL_TEMPERATURE")),
    max_tokens=int(os.getenv("OPENAI_CHAT_MODEL_MAX_TOKENS")),
)

In [91]:
data = {
    "events": [
        "Hiking",
        "Sprint Meeting",
        "RTW 1211",
    ],
    "weather":
        {
            "general": "few clouds",
            "is_dew": "False",
            "is_rain": "False",
            "is_snow": "False"
        },
    "daytime": "midday",
    "art_style": "oil painting"
}

In [92]:
examples = [
    {
        "data": """{
    "events": [
        "Kickoff Meeting",
        "DE10CBA3",
        "Cycling Tour",
        "Band Practice",
    ],
    "weather": {
            "general": "sunny",
            "is_dew": "False",
            "is_rain": "False",
            "is_snow": "False"
        },
    "daytime": "morning",
    "art_style": "photograph"
}""",
        "image_prompt": "A photograph of a person on a bicycle, driving on a road in the woods, on a sunny morning.",

    }
]

In [93]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "data: {data}"),
    ("ai", "image prompt: {image_prompt}")
])

In [94]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [95]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Please generate a prompt for an image generation AI that visualizes the provided data (in JSON format)."),
    few_shot_prompt,
    ("human", "data: {data}"),
    ("ai", "image prompt: ")
])

prompt = final_prompt.format(data=json.dumps(data, indent=4))
print(prompt)

System: Please generate a prompt for an image generation AI that visualizes the provided data (in JSON format).
Human: data: {
    "events": [
        "Kickoff Meeting",
        "DE10CBA3",
        "Cycling Tour",
        "Band Practice",
    ],
    "weather": {
            "general": "sunny",
            "is_dew": "False",
            "is_rain": "False",
            "is_snow": "False"
        },
    "daytime": "morning",
    "art_style": "photograph"
}
AI: image prompt: A photograph of a person on a bicycle, driving on a road in the woods, on a sunny morning.
Human: data: {
    "events": [
        "Hiking",
        "Sprint Meeting",
        "RTW 1211"
    ],
    "weather": {
        "general": "few clouds",
        "is_dew": "False",
        "is_rain": "False",
        "is_snow": "False"
    },
    "daytime": "midday",
    "art_style": "oil painting"
}
AI: image prompt: 


In [96]:
chain = final_prompt | language_model | StrOutputParser()
chain.invoke({"data": data})


'An oil painting of a group of people hiking on a trail with a few clouds in the sky at midday.'